## Aggregation
Data aggregation is the process where raw data is gathered and expressed in a summary form for statistical analysis.

For example, raw data can be aggregated over a given time period to provide statistics such as average, minimum, maximum, sum, and count. After the data is aggregated and written to a view or report, you can analyze the aggregated data to gain insights about particular resources or resource groups.

In [1]:
import numpy as np
import pandas as pd

### Built-in aggregation methods in Pandas
An aggregation method takes a Series of values and returns a single value.

In [33]:
numbers = pd.Series(np.random.randint(low=1, high=100, size=5))
numbers[3] = np.nan
numbers

0    59.0
1    15.0
2    91.0
3     NaN
4    15.0
dtype: float64

In [34]:
# Some of Pandas aggregation methods are:
print(f"{numbers.min() = }")
print(f"{numbers.max() = }")
print(f"{numbers.median() = }")
print(f"{numbers.mean() = }")
print(f"{numbers.sum() = }")
print(f"{numbers.sum() / numbers.count() = }")
print(f"{numbers.mode()[0] = }")
print(f"{numbers.count() = }")
print(f"{numbers.size = }")


numbers.min() = 15.0
numbers.max() = 91.0
numbers.median() = 37.0
numbers.mean() = 45.0
numbers.sum() = 180.0
numbers.sum() / numbers.count() = 45.0
numbers.mode()[0] = 15.0
numbers.count() = 4
numbers.size = 5


### When run on a dataframe (multiple Series) they return a single value for each Series, forming a new Series.

In [40]:
numbers_df = pd.DataFrame(np.random.randint(low=1, high=100, size=[5, 5]), columns=["A", "B", "C", "D", "E"])
numbers_df.loc[[0, 3], ["B", "E"]] = np.nan
numbers_df

,A,B,C,D,E
0,81,NaN,4,46,NaN
1,62,39.0,4,51,72.0
2,62,78.0,24,87,56.0
3,26,NaN,31,55,NaN
4,5,68.0,62,75,26.0


In [37]:
numbers_df.min() # default axis = "index"
numbers_df.min(axis="columns")
numbers_df.min(axis="columns").min()
numbers_df.median(axis="index").median()
numbers_df.median(axis="columns").median()

56.0

In [43]:
# Count null values in each column
numbers_df.isna().sum(axis="index")

A    0
B    2
C    0
D    0
E    2
dtype: int64

### Working with real data

In [44]:
autos = pd.read_json("../Data/autos.json")
autos.tail(3)

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
202,std,sedan,3.58,18,8.8,3012,rwd,front,173,ohcv,...,volvo,95.0,six,four,5500.0,21485.0,2.87,-1,109.1,68.9
203,turbo,sedan,3.01,26,23.0,3217,rwd,front,145,ohc,...,volvo,95.0,six,four,4800.0,22470.0,3.40,-1,109.1,68.9
204,turbo,sedan,3.78,19,9.5,3062,rwd,front,141,ohc,...,volvo,95.0,four,four,5400.0,22625.0,3.15,-1,109.1,68.9


In [45]:
autos["price"].mean()

13207.129353233831

In [47]:
autos[["length", "width", "height"]].head(3)

,length,width,height
0,168.8,64.1,48.8
1,168.8,64.1,48.8
2,171.2,65.5,52.4


In [78]:
# Running mean() on multiple columns (DateFrame) returns a Series of means.
autos[["length", "width", "height"]].head(3).mean()
autos[["length", "width", "height"]].mean()
autos[autos["make"] == "volvo"][["length", "width", "height"]].mean()
autos.query("make=='toyota'")[["length", "width", "height"]].mean()

length    171.934375
width      65.090625
height     53.721875
dtype: float64

### Multiple aggregation

In [76]:
autos[["length", "width", "height"]].agg(["min", "mean", "max"])
#autos.info()
#autos.agg(["min", "mean", "max"])
autos[[col for col in autos.columns if autos[col].dtype in ["int64", "float64"]]].agg(["min", "mean", "max"])

,bore,city-mpg,compression-ratio,curb-weight,engine-size,height,highway-mpg,horsepower,length,normalized-losses,peak-rpm,price,stroke,symboling,wheel-base,width
min,2.540000,13.000000,7.000000,1488.000000,61.000000,47.800000,16.00000,48.000000,141.100000,65.0,4150.000000,5118.000000,2.070000,-2.000000,86.600000,60.300000
mean,3.329751,25.219512,10.142537,2555.565854,126.907317,53.724878,30.75122,104.256158,174.049268,122.0,5125.369458,13207.129353,3.255423,0.834146,98.756585,65.907805
max,3.940000,49.000000,23.000000,4066.000000,326.000000,59.800000,54.00000,288.000000,208.100000,256.0,6600.000000,45400.000000,4.170000,3.000000,120.900000,72.300000


In [ ]:
autos.describe()

### Split-Apply-Combine
- Select a feature to use as key
- Split the dataset into group for each unique key value
- Apply aggregation to each group
- Combine aggregated data into a new dataset

![image](https://nicholasvadivelu.com/assets/images/posts/groupby/split-apply-combine.svg#center)

### Group by
Use Pandas .groupby() method to select a key and split into groups.

This creates a new DataFrameGroupBy oject containing the grouped DataFrames.

In [79]:
makes = autos.groupby("make")

In [80]:
type(makes)

pandas.core.groupby.generic.DataFrameGroupBy

In [81]:
#autos["make"].unique()
#autos["make"].nunique()
#autos["make"].value_counts()
len(makes)

22

In [91]:
makes.groups
autos.loc[makes.groups["jaguar"]]
makes.get_group("jaguar")

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
47,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,145.0,six,four,4750.0,32250.0,4.17,0,113.0,69.6
48,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,NaN,six,four,4750.0,35550.0,4.17,0,113.0,69.6
49,std,sedan,3.54,13,11.5,3950,rwd,front,326,ohcv,...,jaguar,NaN,twelve,two,5000.0,36000.0,2.76,0,102.0,70.6


### Apply & Combine
It's possible to access a single group as above.

However, most of the time we rather apply aggregation functions to each group individually and combine the result into a new dataset.

In [100]:
makes.count().head(3)
makes[["length", "width", "height"]].mean().head(3)

,length,width,height
make,,,
alfa-romero,169.600000,64.566667,50.000000
audi,183.828571,68.714286,54.428571
bmw,184.500000,66.475000,54.825000


### SeriesGroupBy
Indexing a DataFrameGroupBy object with a single column will return a SeriesGroupBy object.

In [114]:
sgb = makes["price"]
type(sgb)
sgb.groups
sgb.get_group("jaguar")
sgb.mean().head(3)

autos.groupby("make")["price"].mean().head(3)


make
alfa-romero    15498.333333
audi           17859.166667
bmw            26118.750000
Name: price, dtype: float64

### Multiple aggregations on SeriesGroupBy
Use pandas .agg() method on SeriesGroupBy to do multiple aggregation on a single feature.

In [119]:
sgb.min()
sgb.agg(["min", "mean", "max"])
autos.groupby("make")["price"].agg(["min", "mean", "max"])
autos.groupby("make")["price"].describe().transpose()

make,alfa-romero,audi,bmw,chevrolet,dodge,honda,isuzu,jaguar,mazda,mercedes-benz,...,nissan,peugot,plymouth,porsche,renault,saab,subaru,toyota,volkswagen,volvo
count,3.000000,6.000000,8.000000,3.000000,9.000000,13.000000,2.000000,3.000000,17.000000,8.000000,...,18.000000,11.000000,7.000000,4.000000,2.000000,6.000000,12.000000,32.000000,12.000000,11.000000
mean,15498.333333,17859.166667,26118.750000,6007.000000,7875.444444,8184.692308,8916.500000,34600.000000,10652.882353,33647.000000,...,10415.666667,15489.090909,7963.428571,31400.500000,9595.000000,15223.333333,8541.250000,9885.812500,10077.500000,18063.181818
std,1734.937559,3452.379493,9263.832033,754.421633,2213.386044,2061.672112,3014.396208,2047.559523,3975.682094,6789.560306,...,4477.394200,2246.749673,2395.544257,6528.784343,424.264069,2860.794761,1940.191468,3204.982114,2178.549872,3314.650263
min,13495.000000,13950.000000,16430.000000,5151.000000,5572.000000,5399.000000,6785.000000,32250.000000,5195.000000,25552.000000,...,5499.000000,11900.000000,5572.000000,22018.000000,9295.000000,11850.000000,5118.000000,5348.000000,7775.000000,12940.000000
25%,14997.500000,15800.000000,19958.750000,5723.000000,6377.000000,6855.000000,7850.750000,33900.000000,7395.000000,28230.000000,...,7311.500000,13530.000000,6460.500000,29900.500000,9445.000000,12887.500000,7378.750000,7870.500000,8145.000000,16250.000000
50%,16500.000000,17580.000000,22835.000000,6295.000000,7609.000000,7295.000000,8916.500000,35550.000000,10595.000000,32892.000000,...,8124.000000,16630.000000,7609.000000,33278.000000,9595.000000,15275.000000,7894.000000,9103.000000,9737.500000,18420.000000
75%,16500.000000,18617.500000,32290.000000,6435.000000,8558.000000,9095.000000,9982.250000,35775.000000,11845.000000,36532.000000,...,13499.000000,16987.500000,8439.000000,34778.000000,9745.000000,17490.000000,10019.500000,10973.250000,11768.750000,20265.000000
max,16500.000000,23875.000000,41315.000000,6575.000000,12964.000000,12945.000000,11048.000000,36000.000000,18344.000000,45400.000000,...,19699.000000,18150.000000,12764.000000,37028.000000,9895.000000,18620.000000,11694.000000,17669.000000,13845.000000,22625.000000


### Multiple aggregation on DataFrameGroupBy
Using pandas .agg() method on a DataFrameGroupBy to do multiple aggregation on multiple features will return a multi-index column dataframe.

In [121]:
makes[["length", "width", "height"]].agg(["min", "mean", "max"]).head(3)

length                    width                  height  \
               min        mean    max   min       mean   max    min   
make                                                                  
alfa-romero  168.8  169.600000  171.2  64.1  64.566667  65.5   48.8   
audi         176.6  183.828571  192.7  66.2  68.714286  71.4   52.0   
bmw          176.8  184.500000  197.0  64.8  66.475000  70.9   53.7   

                              
                  mean   max  
make                          
alfa-romero  50.000000  52.4  
audi         54.428571  55.9  
bmw          54.825000  56.3

### Custom columns aggregation

In [123]:
autos.groupby("make").agg({"price": "mean", "horsepower": "max"}).head(3)

,price,horsepower
make,,
alfa-romero,15498.333333,154.0
audi,17859.166667,160.0
bmw,26118.750000,182.0


In [166]:
def list_unique(x):
    return ", ".join(sorted(x.apply(str).unique()))

autos.groupby("make").agg(
    average_price = pd.NamedAgg(column="price", aggfunc="mean"),
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min"),
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max"),
    body_styles = pd.NamedAgg(column="body-style", aggfunc=list_unique)
    #body_styles = pd.NamedAgg(column="body-style", aggfunc=lambda x: ", ".join(sorted(x.apply(str).unique())))    
).sort_values("average_price", ascending=False).head()

,average_price,min_horsepower,max_horsepower,body_styles
make,,,,
jaguar,34600.000000,176.0,262.0,sedan
mercedes-benz,33647.000000,123.0,184.0,"convertible, hardtop, sedan, wagon"
porsche,31400.500000,143.0,288.0,"convertible, hardtop, hatchback"
bmw,26118.750000,101.0,182.0,sedan
volvo,18063.181818,106.0,162.0,"sedan, wagon"


In [158]:
#def my_sorting_func(person):
#    return len(person["FirstName"])

mylist = [
    {"FirstName": "Fredrik", "LastName": "Johansson", "Age": 42},
    {"FirstName": "Anders", "LastName": "Svensson", "Age": 22},
    {"FirstName": "Maria", "LastName": "Karlsson", "Age": 27},
    {"FirstName": "Anna", "LastName": "Bengtsson", "Age": 39},


]
sorted(mylist, key=lambda person: len(person["FirstName"]), reverse=True)


[{'FirstName': 'Fredrik', 'LastName': 'Johansson', 'Age': 42},
 {'FirstName': 'Anders', 'LastName': 'Svensson', 'Age': 22},
 {'FirstName': 'Maria', 'LastName': 'Karlsson', 'Age': 27},
 {'FirstName': 'Anna', 'LastName': 'Bengtsson', 'Age': 39}]

In [163]:
def myfunc():
    return "Hello"

myfunc = lambda: "Hello"


otherfunc = myfunc

print(otherfunc())

Hello
